In [26]:
import gpflow as gpf
import numpy as np
import gpflow.multioutput.kernels as mk
import gpflow.multioutput.features as mf
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from time import time
import tensorflow as tf
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import os
# from latent_constraint_learning import experiments_util
tf.logging.set_verbosity(tf.logging.FATAL)
plt.style.use("slides")
np.set_printoptions(suppress=True, precision=4, linewidth=200)

In [8]:
def format_forward_data(data, traj_idx_start=0, traj_idx_end=-1, take_every=1):
    states = data['states'][traj_idx_start:traj_idx_end]
    for traj_idx  in range(states.shape[0]):
        states[traj_idx, :, 0] -= states[traj_idx, 0, 4]
        states[traj_idx, :, 1] -= states[traj_idx, 0, 5]
        states[traj_idx, :, 2] -= states[traj_idx, 0, 4]
        states[traj_idx, :, 3] -= states[traj_idx, 0, 5]
        states[traj_idx, :, 4] -= states[traj_idx, 0, 4]
        states[traj_idx, :, 5] -= states[traj_idx, 0, 5]
    actions = data['actions'][traj_idx_start:traj_idx_end]
    state_dim = states.shape[2]
    action_dim = actions.shape[2]
    s = states[:, :-take_every, :]
    s_next = states[:, take_every:, :]
    s_flat = s.reshape(-1, state_dim)
    s_next_flat = s_next.reshape(-1, state_dim)
    if take_every == 1:
        u_flat = actions[:, :, :].reshape(-1, action_dim)
    else:
        u_flat = actions[:, :-(take_every-1), :].reshape(-1, action_dim)
    
    combined_x = np.concatenate((s_flat, u_flat), axis=1)
    # make data more zero centered by making things relative to the head
    # input is [x_tail - x_head, y_tail - y_head, x_mid - x_head, y_mid - y_head, 0, 0, vx_head, vy_head]
    # output is [delta x_tail, delta y_tail, delt x_mid, delta y_mid, delta x_head, delta_y_head]
    x_flat = s_flat
    # predict the delta only
    y_flat = s_next_flat - s_flat
    # when doing rollouts, first you have to take the 
    x_trajs = s
    u_trajs = actions
    return x_flat, y_flat, u_flat, combined_x, x_trajs, u_trajs

def format_inverse_data(data, traj_idx_start=0, traj_idx_end=-1, take_every=1):
    states = data['states'][traj_idx_start:traj_idx_end]
    for traj_idx  in range(states.shape[0]):
        states[traj_idx, :, 0] -= states[traj_idx, 0, 4]
        states[traj_idx, :, 1] -= states[traj_idx, 0, 5]
        states[traj_idx, :, 2] -= states[traj_idx, 0, 4]
        states[traj_idx, :, 3] -= states[traj_idx, 0, 5]
        states[traj_idx, :, 4] -= states[traj_idx, 0, 4]
        states[traj_idx, :, 5] -= states[traj_idx, 0, 5]
    actions = data['actions'][traj_idx_start:traj_idx_end]
    state_dim = states.shape[2]
    action_dim = actions.shape[2]
    s = states[:, :-take_every, :]
    s_next = states[:, take_every:, :]
    s_flat = s.reshape(-1, state_dim)
    s_next_flat = s_next.reshape(-1, state_dim)
    if take_every == 1:
        u_flat = actions[:, :, :].reshape(-1, action_dim)
    else:
        u_flat = actions[:, :-(take_every-1), :].reshape(-1, action_dim)
    
    combined_x = np.concatenate((s_flat, s_next_flat - s_flat), axis=1)
    return combined_x, u_flat

In [9]:
data = np.load("240_50_random4.npz")
print('number of trajectories: {}'.format(data['states'].shape[0]))
train_idx_start = 0
train_idx_end = 200
test_idx_start = 200
test_idx_end = 240 
take_every = 1

train_x_flat, train_y_flat, train_u_flat, combined_train_x, train_x_trajs, train_u_trajs = format_forward_data(data, train_idx_start, train_idx_end, take_every)
test_x_flat, test_y_flat, test_u_flat, combined_test_x, test_x_trajs, test_u_trajs = format_forward_data(data, test_idx_start, test_idx_end, take_every)
train_inverse_combined_x, train_inverse_y = format_inverse_data(data, train_idx_start, train_idx_end, take_every)
test_inverse_combined_x, test_inverse_y = format_inverse_data(data, test_idx_start, test_idx_end, take_every)

number of trajectories: 240


In [10]:
class MyModel:
    
    def __init__(self, X, Y, M=100):
        self.N, self.D = X.shape
        _, self.P = Y.shape
        self.M = M # number of inducing points
        self.X = X
        self.Y = Y

        self.model_def = {
            'class': gpf.kernels.SquaredExponential,
            'initial_hyper_params': {
                'lengthscales': 1.0
            },
            'initial_likelihood_variance': [0.1]*self.P
        }
        self.kern_list = [self.model_def['class'](self.D, **self.model_def['initial_hyper_params']) for _ in range(self.P)]
        self.kernel = mk.SeparateIndependentMok(self.kern_list)

        self.Zs = [X[np.random.permutation(self.N)[:self.M],...].copy() for _ in range(self.P)]
        # initialise as list inducing features
        self.feature_list = [gpf.features.InducingPoints(Z) for Z in self.Zs]
        # create multioutput features from feature_list
        self.feature = mf.SeparateIndependentMof(self.feature_list)

        self.likelihood_variance = self.model_def['initial_likelihood_variance']
        self.likelihood = gpf.likelihoods.Gaussian(self.likelihood_variance)
        self.model = gpf.models.SVGP(self.X, self.Y, self.kernel, self.likelihood, feat=self.feature)
        self.opt = gpf.train.ScipyOptimizer()

    def train(self, print_time=True, display=True, maxiter=300):
        t0 = time()
        self.opt.minimize(self.model, disp=display, maxiter=maxiter)
        dt = time() - t0
        if print_time:
            print("training time: {}s".format(dt))
            
    def metadata(self):
        return {
            'N': self.N,
            'D': self.D,
            'P': self.P,
            'M': self.M,
            'kernel_type': self.model_def['class'].__name__,
            'initial_hyper_params': self.model_def['initial_hyper_params'],
            'initial_likelihood_variance': self.model_def['initial_likelihood_variance'],
        }

In [11]:
gpf.reset_default_graph_and_session()
fwd_model = MyModel(combined_train_x, train_y_flat)
inv_model = MyModel(train_inverse_combined_x, train_inverse_y)

In [12]:
maxiter = 100
fwd_model.train(maxiter=maxiter)

training time: 30.246812343597412s


In [13]:
saver = gpf.saver.Saver()
metadata = {
    'maxiter': maxiter,
}
# metadata.update(fwd_model.metadata())
# log_path = experiments_util.experiment_name('separate_independant', 'gpf')
# full_log_path = os.path.join(os.getcwd(), 'log_data', log_path)
# experiments_util.make_log_dir(full_log_path)
# experiments_util.write_metadata(metadata, log_path)
# model_path = os.path.join(full_log_path, 'forward_model')
# saver.save(model_path, fwd_model.model)

In [14]:
# pretty sure we don't need to learn this inverse model, we can use non-linear optimization to solve this with the forward model?
inv_model.train(maxiter=100)

training time: 10.653449535369873s


In [15]:
def mean_total_node_position_error(my_model, test_x, test_y):
    """ compute the euclidian distance for each node in pred_y[i] to each node in test_y[i], averaged over all i using the max likelihood prediction"""
    pred_delta_x_mean, pred_delta_x_sigma = my_model.model.predict_y(test_x)
    tail_error = np.linalg.norm(pred_delta_x_mean[:, 0:2] - test_y[:, 0:2], axis=1)
    mid_error = np.linalg.norm(pred_delta_x_mean[:, 2:4] - test_y[:, 2:4], axis=1)
    head_error = np.linalg.norm(pred_delta_x_mean[:, 4:6] - test_y[:, 4:6], axis=1)
    print('tail position error (min/max/mean)', tail_error.min(), tail_error.max(), tail_error.mean())
    print('mid position error (min/max/mean)', mid_error.min(), mid_error.max(), mid_error.mean())
    print('head position error (min/max/mean)', head_error.min(), head_error.max(), head_error.mean())
    total_node_error = tail_error + mid_error + head_error
    print('min total error', total_node_error.min())
    print('max total error', total_node_error.max())
    print('mean total error', total_node_error.mean())

def mean_control_error(my_model, test_x, test_y):
    """ compute the euclidian distance between the predicted control and the true control"""
    pred_u_mean, pred_u_sigma = my_model.model.predict_y(test_x)
    pred_speeds = np.linalg.norm(pred_u_mean, axis=1)
    
    speed_error = np.linalg.norm(pred_u_mean - test_y, axis=1)
    
    pred_u_norm = np.linalg.norm(pred_u_mean, axis=1)
    test_y_norm = np.linalg.norm(test_y, axis=1)
    
    # compute dot product of each column of a with each column of b
    angle_error = np.arccos(np.einsum('ij,ij->i', pred_u_mean, test_y) / (pred_u_norm*test_y_norm))

    print('min pred speed', np.min(pred_speeds))
    print('max pred speed', np.max(pred_speeds))
    print('mean pred speed', np.mean(pred_speeds))
    
    print("min speed error = {:0.3f} m/s".format(speed_error.min()))
    print("max speed error = {:0.3f} m/s".format(speed_error.max()))
    print("mean speed error = {:0.3f} m/s".format(speed_error.mean()))
    
    print("min angle error = {:0.3f} m/s".format(angle_error.min()))
    print("max angle error = {:0.3f} m/s".format(angle_error.max()))
    print("mean angle error = {:0.3f} m/s".format(angle_error.mean()))

In [16]:
total_displacement = np.linalg.norm(test_y_flat[:, 0:2], axis=1) + np.linalg.norm(test_y_flat[:, 2:4], axis=1) + np.linalg.norm(test_y_flat[:, 4:6], axis=1)
print('min displacement in test', np.min(total_displacement))
print('max displacement in test', np.max(total_displacement))
print('mean displacement in test', np.mean(total_displacement))

min displacement in test 0.0010301430513861287
max displacement in test 0.30061229261431177
mean displacement in test 0.11060115018261646


In [17]:
# error for trained model
mean_total_node_position_error(fwd_model, combined_test_x, test_y_flat)

tail position error (min/max/mean) 0.0002787365327200829 0.09368788255045733 0.024556553951345116
mid position error (min/max/mean) 0.0008177865214450011 0.08580572216234945 0.021676008794372283
head position error (min/max/mean) 0.00026787803611576553 0.057631217648513165 0.00663875795703223
min total error 0.007760264031475282
max total error 0.17160683798073945
mean total error 0.05287132070274963


In [18]:
# check that the statistics of the control predictions match the training data
print("min/max/mean/median of control input:")
train_speeds = np.linalg.norm(train_inverse_y, axis=1)
print("=====training=====")
print(np.min(train_speeds))
print(np.max(train_speeds))
print(np.mean(train_speeds))
print(np.median(train_speeds))
test_speeds = np.linalg.norm(test_inverse_y, axis=1)
print("=====testing=====")
print(np.min(test_speeds))
print(np.max(test_speeds))
print(np.mean(test_speeds))
print(np.median(test_speeds))
print("=====prediction=====")
mean_control_error(inv_model, test_inverse_combined_x, test_inverse_y)

min/max/mean/median of control input:
=====training=====
0.0027032138935026984
0.9903389473967043
0.47104272124346014
0.43892548204609216
=====testing=====
0.0782460996904688
0.9903685221395796
0.5768620286866841
0.6301570903618964
=====prediction=====
min pred speed 0.0027983151681439038
max pred speed 1.0509858146649491
mean pred speed 0.5070520906323781
min speed error = 0.002 m/s
max speed error = 0.717 m/s
mean speed error = 0.149 m/s
min angle error = 0.000 m/s
max angle error = 2.526 m/s
mean angle error = 0.166 m/s


In [19]:
def predict(fwd_model, traj, steps=1, initial_variance = 0.00001):
    test_x_traj, test_u_traj  = traj
    traj_length = test_x_traj.shape[0]
    
    assert steps < traj_length, "steps {} > traj length {}!".format(steps, traj_length)
    
    if test_x_traj.shape[0] > test_u_traj.shape[0]:
        test_x_traj = test_x_traj[:test_u_traj.shape[0]]
        
    mu_combined_test_x = np.hstack((test_x_traj, test_u_traj))
        
    # define the initial distribution
    mu_combined_test_x_t = mu_combined_test_x[0]
    sigma_combined_test_x = np.eye(fwd_model.D) * initial_variance
    # assume no control variance
    sigma_combined_test_x[-2, -2] = 0
    sigma_combined_test_x[-1, -1] = 0
    
    # sample from that initial distribution to get initial particles to feed into the GP
    num_particles = 50
    combined_x_t_particles = np.random.multivariate_normal(mu_combined_test_x_t, sigma_combined_test_x, num_particles)
    particles = np.zeros((steps, num_particles, fwd_model.D))
    
    for t in range(steps):
        particles[t] = combined_x_t_particles
        mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s = fwd_model.model.predict_y(combined_x_t_particles)
        # sample point from the gaussian prediction
        combined_x_t_plus_1_particles = np.empty_like(combined_x_t_particles)
        for j, (mu_delta_x_t_plus_1_j, var_delta_x_t_plus_1_j) in enumerate(zip(mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s)):
            # We assumed that the GPs are independant for each output dimension, so the full covariance matrix is diagonal
            sigma_delta_x_t_plus_1_j = np.diag(var_delta_x_t_plus_1_j)
            u_t_plus_1_j = test_u_traj[t + 1]
            delta_delta_x_t_j = np.random.multivariate_normal(mu_delta_x_t_plus_1_j, sigma_delta_x_t_plus_1_j)
            # predict only gives the delta position, so we have to integrate here
            delta_delta_combined_x_t_j = np.hstack((delta_delta_x_t_j, [0,0 ]))
            combined_x_t_plus_1_particles[j] =  combined_x_t_particles[j] + delta_delta_combined_x_t_j
        
        combined_x_t_particles = combined_x_t_plus_1_particles
            
    return particles, test_x_traj, test_u_traj

In [20]:
def predict_no_ground_truth(my_model, initial_x, u, steps=1, initial_variance = 0.00001):
    # define the initial distribution
    mu_combined_x_t =  np.hstack((initial_x, u))
    sigma_combined_x_t = np.eye(my_model.D) * initial_variance
    # assume no control variance
    sigma_combined_x_t[-2, -2] = 0
    sigma_combined_x_t[-1, -1] = 0
    
    # sample from that initial distribution to get initial particles to feed into the GP
    num_particles = 5
    combined_x_t_particles = np.random.multivariate_normal(mu_combined_x_t, sigma_combined_x_t, num_particles)
    print(combined_x_t_particles)
    particles = np.zeros((steps, num_particles, my_model.D))
    
    for t in range(steps):
        particles[t] = combined_x_t_particles
        mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s = my_model.model.predict_y(combined_x_t_particles)
        # sample point from the gaussian prediction
        combined_x_t_plus_1_particles = np.empty_like(combined_x_t_particles)
        for j, (mu_delta_x_t_plus_1_j, var_delta_x_t_plus_1_j) in enumerate(zip(mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s)):
            # We assumed that the GPs are independant for each output dimension, so the full covariance matrix is diagonal
            sigma_delta_x_t_plus_1_j = np.diag(var_delta_x_t_plus_1_j)
            delta_delta_x_t_j = np.random.multivariate_normal(mu_delta_x_t_plus_1_j, sigma_delta_x_t_plus_1_j)
            # predict only gives the delta position, so we have to integrate here
            delta_delta_combined_x_t_j = np.hstack((delta_delta_x_t_j, [0,0 ]))
            combined_x_t_plus_1_particles[j] =  combined_x_t_particles[j] + delta_delta_combined_x_t_j
        
        combined_x_t_particles = combined_x_t_plus_1_particles
            
    return particles

In [21]:
%matplotlib inline 

def plot_predict(particles, test_x_traj=None, test_u_traj=None):
    T = particles.shape[0]
    if test_x_traj is not None:
        x0s = [test_x_traj[0, 0], test_x_traj[0, 2], test_x_traj[0, 4]]
        y0s = [test_x_traj[0, 1], test_x_traj[0, 3], test_x_traj[0, 5]]
    
    fig = plt.figure(figsize=(10,10))
    
    x_t_particles_data = []
    for x_t_particle in particles[0]:
        x_t_particles_xs = [x_t_particle[0], x_t_particle[2], x_t_particle[4]]
        x_t_particles_ys = [x_t_particle[1], x_t_particle[3], x_t_particle[5]]
        line = plt.plot(x_t_particles_xs, x_t_particles_ys, color='black', alpha=0.4)[0]
        x_t_particles_data.append(line)
            
    
    x_next_data = []
    if test_x_traj is not None:
        plt.plot(x0s, y0s, color='orange')
        x_t_plus_1 = test_x_traj[1]
        xs_next = [x_t_plus_1[0], x_t_plus_1[2], x_t_plus_1[4]]
        ys_next = [x_t_plus_1[1], x_t_plus_1[3], x_t_plus_1[5]]
        x_next_data = plt.plot(xs_next, ys_next, color='blue')[0]

    if test_x_traj is not None:
        plt.quiver(test_x_traj[0, 4], test_x_traj[0, 5], test_u_traj[0, 0], test_u_traj[0, 1], color='black')

    plt.xlabel("x (m)")
    plt.ylabel("y (m)")
    plt.xlim([-8,8])
    plt.ylim([-8,8])
    custom_lines = [Line2D([0], [0], color='black', lw=1, alpha=0.1),
                    Line2D([0], [0], color='orange', lw=1),
                    Line2D([0], [0], color='blue', lw=1)]

    l = plt.legend(custom_lines, ['predictions', r'$x_0$', r'$x_t$'])
    
    def update(t):
        x_t_particles = particles[t]
        for x_t_particle_d, x_t_particle in zip(x_t_particles_data, x_t_particles):
            x_t_particles_xs = [x_t_particle[0], x_t_particle[2], x_t_particle[4]]
            x_t_particles_ys = [x_t_particle[1], x_t_particle[3], x_t_particle[5]]
            x_t_particle_d.set_xdata(x_t_particles_xs)
            x_t_particle_d.set_ydata(x_t_particles_ys)
        
        if test_x_traj is not None:
            x_t_plus_1 = test_x_traj[t]
            xs_next = [x_t_plus_1[0], x_t_plus_1[2], x_t_plus_1[4]]
            ys_next = [x_t_plus_1[1], x_t_plus_1[3], x_t_plus_1[5]]
            x_next_data.set_xdata(xs_next)
            x_next_data.set_ydata(ys_next)
        ax = fig.gca()
        ax.relim()
        ax.autoscale_view()

    
    anim = FuncAnimation(fig, update, frames=np.arange(0, T), interval=100)
    plt.close()
    return anim

In [27]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

In [33]:
np.random.seed(1)
# 9 and 13 are interesting
# 14 is accurate
trajectory_idx = 3
traj = test_x_trajs[trajectory_idx], test_u_trajs[trajectory_idx]
plot_args = predict(fwd_model, traj, steps=49)
anim = plot_predict(*plot_args)
anim_html = anim.to_jshtml()
HTML(anim.to_jshtml())
anim.save('test_ex_{}.gif'.format(trajectory_idx), writer='imagemagick', fps=20)

In [23]:
initial_x = np.array([0,1,0,0,0,-1])
u = np.array([0, 0])
plot_args = predict_no_ground_truth(m, initial_x, u, steps=200)
anim = plot_predict(plot_args)
HTML(anim.to_jshtml())

NameError: name 'm' is not defined

In [ ]:
def trajectory_following(fwd_model, inv_model, x_traj, steps=1, initial_variance = 0.00001):
    traj_length = x_traj.shape[0]
    
    assert steps < traj_length, "steps {} > traj length {}!".format(steps, traj_length)
    
    # define the initial distribution
    delta_x_1 = x_traj[1:2] - x_traj[0:1]
    mu_u0, sigma_u0 = inv_model.model.predict_y(np.hstack((x_traj[0:1], delta_x_1)))
    print(mu_u0.shape)
    print(mu_u0, sigma_u0)
    return
    mu_combined_test_x_t = np.hstack((x_traj[0], u0))
    sigma_combined_test_x = np.eye(fwd_model.D) * initial_variance
    # assume no control variance
    sigma_combined_test_x[-2, -2] = 0
    sigma_combined_test_x[-1, -1] = 0
    
    # sample from that initial distribution to get initial particles to feed into the GP
    num_particles = 2
    combined_x_t_particles = np.random.multivariate_normal(mu_combined_test_x_t, sigma_combined_test_x, num_particles)
    particles = np.zeros((steps, num_particles, fwd_model.D))
    
    for t in range(steps):
        # take the next step in the reference trajectory and construct the input to the inverse model
        traj_x_t_plus_1 = test_x_traj[t + 1]
        delta_x_t_particles = combined_x_t_particles - traj_x_t_plus_1
        inv_model_input = np.hstack((combined_x_t_particles, delta_x_t_particles))
        u = inv_model.model.predict_y(inv_model_input)
        print(u.shape)
        
        particles[t] = combined_x_t_particles
        mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s = fwd_model.model.predict_y(combined_x_t_particles)
        # sample point from the gaussian prediction
        combined_x_t_plus_1_particles = np.empty_like(combined_x_t_particles)
        for j, (mu_delta_x_t_plus_1_j, var_delta_x_t_plus_1_j) in enumerate(zip(mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s)):
            # We assumed that the GPs are independant for each output dimension, so the full covariance matrix is diagonal
            sigma_delta_x_t_plus_1_j = np.diag(var_delta_x_t_plus_1_j)
            u_t_plus_1_j = test_u_traj[t + 1]
            delta_delta_x_t_j = np.random.multivariate_normal(mu_delta_x_t_plus_1_j, sigma_delta_x_t_plus_1_j)
            # predict only gives the delta position, so we have to integrate here
            delta_delta_combined_x_t_j = np.hstack((delta_delta_x_t_j, [0,0 ]))
            combined_x_t_plus_1_particles[j] =  combined_x_t_particles[j] + delta_delta_combined_x_t_j
        
        combined_x_t_particles = combined_x_t_plus_1_particles
            
    return particles, test_x_traj, test_u_traj
    

In [ ]:
np.random.seed(1)
test_example_idx = 0
plot_args = trajectory_following(fwd_model, inv_model, test_x_trajs[test_example_idx], steps=4)
print(test_u_trajs[test_example_idx,0])
# anim = plot_predict(*plot_args)
# HTML(anim.to_jshtml())